## Úprava tabulky s cenami v minulých letech

1. import knihoven

In [11]:
import pyodbc
import pandas as pd
import re
from sqlalchemy import create_engine
import sqlalchemy
from urllib.parse import quote_plus

2. napojení do databáze

In [12]:
server = 'daproject.database.windows.net'
database = 'Project_Czechitas'
authentication = 'ActiveDirectoryInteractive'
driver = '{ODBC Driver 17 for SQL Server}'
connect_args = {'charset': 'utf8mb4'}
params = quote_plus('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';Authentication='+authentication+';TrustServerCertificate='+'no')
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", connect_args=connect_args)

3. nahrání dat do DataFrame

In [13]:
query = 'select * from flat_rent'
sales = pd.read_sql_query(query, engine)
sales

,date,price,price_per_m2
0,leden 2015,11308,178
1,únor 2015,11357,180
2,březen 2015,11491,186
3,duben 2015,11676,187
4,květen 2015,11820,188
...,...,...,...
100,květen 2023,17672,336
101,červen 2023,17969,352
102,červenec 2023,17709,366
103,srpen 2023,18090,380


4. odstranění mezer

In [14]:
sales['date'] = sales['date'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
sales

,date,price,price_per_m2
0,leden 2015,11308,178
1,únor 2015,11357,180
2,březen 2015,11491,186
3,duben 2015,11676,187
4,květen 2015,11820,188
...,...,...,...
100,květen 2023,17672,336
101,červen 2023,17969,352
102,červenec 2023,17709,366
103,srpen 2023,18090,380


5. vytvoření slovníku pro exportované datové typy (bez NVARCHAR se stále exportovalo bez diakritiky)

In [15]:
output = {
    'date': sqlalchemy.NVARCHAR(),
    'price': sqlalchemy.types.SmallInteger(),
    'price_per_m2': sqlalchemy.types.SmallInteger()
}

6. zápis do databáze

In [16]:
sales.to_sql('flat_rent', con=engine, if_exists='replace', index=False, dtype = output)

-1